# Exploring Movie Industry Insights for Microsoft Movie Studio

## Overview

Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. We are charged with exploring what types of films are currently doing the best at the box office. We must then translate those findings into actionable insights that the head of Microsoft's new movie studio can use to help decide what type of films to create.

## Business Problem

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas in vulputate orci, eget posuere massa. Cras at lectus vel diam porta viverra. Ut vehicula congue aliquam. Suspendisse finibus risus quis hendrerit rhoncus. Sed euismod semper hendrerit. Sed nec vulputate diam.

## Data Understanding

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas in vulputate orci, eget posuere massa. Cras at lectus vel diam porta viverra. Ut vehicula congue aliquam. Suspendisse finibus risus quis hendrerit rhoncus. Sed euismod semper hendrerit. Sed nec vulputate diam.

In [5]:
import pandas as pd
import numpy as np
import sqlite3

In [4]:
# We will be working primairly with three datasets.
# The first are easy to import since they are stored as files with comma-seperated values
box_office = pd.read_csv('data/bom.movie_gross.csv')
movie_budgets = pd.read_csv('data/tn.movie_budgets.csv')

In [6]:
# The other data source is a SQLite database from the Internet Movie Database (IMDB).
conn = sqlite3.connect('data/im.db')

In [7]:
box_office.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [8]:
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


### The IMDB Data
The IMDB data is composed of 8 different tables. Below is its ERD.

![IMDB ERD](images/imdb_erd.jpeg)

This dataset is the most roboust and likely contains more information than we need. A quick look at the movie_basics table shows us that the dataset contains information on 146,144 unique records between the years of 2010 and 2115 (which is either an error or at the very least something to look into when we do data cleaning).

Looking at the other tables in the databse, it looks like we also have information on each movie's: staff (actors, producers, writers, etc.), names of chartacters played by actors, some basic information on the people involved (like birth year and year of death), as well as info on movie ratings.

In [31]:
q = """
SELECT MIN(start_year), MAX(start_year), COUNT(DISTINCT movie_id)
FROM movie_basics
;"""

pd.read_sql(q, conn)

,MIN(start_year),MAX(start_year),COUNT(DISTINCT movie_id)
0,2010,2115,146144


In [33]:
q = """
SELECT *
FROM principals
GROUP BY category
LIMIT 10
;"""

pd.read_sql(q, conn)

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"
2,tt1179035,1,nm3743659,archive_footage,None,"[""Himself""]"
3,tt1754621,4,nm5080976,archive_sound,None,"[""Himself (videographer)""]"
4,tt0323808,9,nm0676104,cinematographer,None,None
5,tt0323808,8,nm0779346,composer,None,None
6,tt0111414,2,nm0398271,director,None,None
7,tt0323808,10,nm0059247,editor,None,None
8,tt0111414,3,nm3739909,producer,producer,None
9,tt10229602,10,nm3193762,production_designer,None,None


### Box Office Data

The box office comes from Box Office Mojo. It contains records on 3,386 unique movie titles between the years of 2010 and 2018. For each movie, it provides us with the movie's title, the studio which produced it, the domestic box office gross earnings, foreign box office gross earnings and the year of release.

In [15]:
box_office.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [17]:
# Get the range of years in the box office dataset

print('The data sets begins in the year:', box_office['year'].min())
print('The dataset contains data on movies through the year: ', box_office['year'].max())

The data sets begins in the year: 2010
The dataset contains data on movies through the year:  2018


In [21]:
# How many unique movies titles are in the dataset?
box_office['title'].nunique()

3386

### Movie Budgets Data

The movie budgets data comes from The-Numbers.com. It contains records on 5,698 unique movie titles between the years of 1915 and 2020. For each movie, it provides us with the movie's release date, its title, its production budget, and its foreign and domestic box office earnings.

In [22]:
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [23]:
# How many unique movies titles are in the dataset?
movie_budgets['movie'].nunique()

5698

In [27]:
# Get the range of years in the box office dataset

movie_budgets['year'] = movie_budgets['release_date'].str[-4:]

print('The data sets begins in the year:', movie_budgets['year'].min())
print('The dataset contains data on movies through the year: ', movie_budgets['year'].max())

The data sets begins in the year: 1915
The dataset contains data on movies through the year:  2020


## Data Preperation

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas in vulputate orci, eget posuere massa. Cras at lectus vel diam porta viverra. Ut vehicula congue aliquam. Suspendisse finibus risus quis hendrerit rhoncus. Sed euismod semper hendrerit. Sed nec vulputate diam.

### Data Cleaning
Lorem ipsum dolor sit amet, consectetur adipiscing elit.

### Merging Data Sets
Lorem ipsum dolor sit amet, consectetur adipiscing elit.

## Analysis

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas in vulputate orci, eget posuere massa. Cras at lectus vel diam porta viverra. Ut vehicula congue aliquam. Suspendisse finibus risus quis hendrerit rhoncus. Sed euismod semper hendrerit. Sed nec vulputate diam.

## Conclusions

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas in vulputate orci, eget posuere massa. Cras at lectus vel diam porta viverra. Ut vehicula congue aliquam. Suspendisse finibus risus quis hendrerit rhoncus. Sed euismod semper hendrerit. Sed nec vulputate diam.

### Next Steps

Lorem ipsum dolor sit amet, consectetur adipiscing elit.

* Maecenas in vulputate orci, eget posuere massa.
* Cras at lectus vel diam porta viverra.
* Ut vehicula congue aliquam. 